# [Expressions: Strings](https://docs.pola.rs/user-guide/expressions/strings/)

## The string namespace

Small note: if there are only ASCII symbols, then `.str.len_bytes().` will be faster.

In [2]:
import polars as pl

df = pl.DataFrame(
    {
        "language": ["English", "Dutch", "Portuguese", "Finish"],
        "fruit": ["pear", "peer", "pêra", "päärynä"],
    }
)

df.with_columns(
    pl.col("fruit").str.len_bytes().alias("byte_count"),
    pl.col("fruit").str.len_chars().alias("letter_count"),
)

language,fruit,byte_count,letter_count
str,str,u32,u32
"""English""","""pear""",4,4
"""Dutch""","""peer""",4,4
"""Portuguese""","""pêra""",5,4
"""Finish""","""päärynä""",10,7


## Parsing strings

### Check for the existence of a pattern

In [6]:
df.select(
    pl.col("fruit"),
    pl.col("fruit").str.starts_with("p").alias("starts_with_p"),
    pl.col("fruit").str.contains("p..r").alias("p..r"),
    pl.col("fruit").str.contains("e+").alias("e+"),
    pl.col("fruit").str.ends_with("r").alias("ends_with_r"), # make note of this
)

fruit,starts_with_p,p..r,e+,ends_with_r
str,bool,bool,bool,bool
"""pear""",true,true,true,true
"""peer""",true,true,true,true
"""pêra""",true,false,false,false
"""päärynä""",true,true,false,false


### Regex specification

polars uses the rust regex synthax [click here for the details](https://docs.rs/regex/latest/regex/#syntax). its diferent from the `re` module.

### Extract a pattern

In [7]:
df = pl.DataFrame(
    {
        "urls": [
            "http://vote.com/ballon_dor?candidate=messi&ref=polars",
            "http://vote.com/ballon_dor?candidat=jorginho&ref=polars",
            "http://vote.com/ballon_dor?candidate=ronaldo&ref=polars",
        ]
    }
)

df.select(
    pl.col("urls").str.extract(r"candidate=(\w+)", group_index=1), # grouping in regex is awesome, I needed to know that years ago.
)

urls
str
"""messi"""
null
"""ronaldo"""


In [10]:
df = pl.DataFrame({"text": ["123 bla 45 asd", "xyz 678 910t"]})
df.select(
    pl.col("text").str.extract_all(r"(\d+)").alias("extracted_nrs"),
)

extracted_nrs
list[str]
"[""123"", ""45""]"
"[""678"", ""910""]"


### replace a pattern

In [ ]:
df = pl.DataFrame({"text": ["123abc", "abc456"]})
df.with_columns(
    pl.col("text").str.replace(r"\d","-"),
    pl.col("text").str.replace_all(r"\d","-").alias("text_replace_all")
)

text,text_replace_all
str,str
"""-23abc""","""---abc"""
"""abc-56""","""abc---"""


## Modifying strings

### Case conversion

In [14]:
addresses = pl.DataFrame(
    {
        "addresses": [
            "128 PERF st",
            "Rust blVD, 158",
            "PoLaRs Av, 12",
            "1042 Query sq",
        ]
    }
)
addresses.select(
    pl.col("addresses").alias("originals"),
    pl.col("addresses").str.to_titlecase(),
    pl.col("addresses").str.to_lowercase().alias("lower"),
    pl.col("addresses").str.to_uppercase().alias("upper"),
)

originals,addresses,lower,upper
str,str,str,str
"""128 PERF st""","""128 Perf St""","""128 perf st""","""128 PERF ST"""
"""Rust blVD, 158""","""Rust Blvd, 158""","""rust blvd, 158""","""RUST BLVD, 158"""
"""PoLaRs Av, 12""","""Polars Av, 12""","""polars av, 12""","""POLARS AV, 12"""
"""1042 Query sq""","""1042 Query Sq""","""1042 query sq""","""1042 QUERY SQ"""


### Stripping characters from the ends

In [19]:
addr = pl.col("addresses") # the bastards made my type all that stuff!
chars = ", 0123456789"
addresses.select(
    addr,
    addr.str.strip_chars(chars).alias("strip"),
    addr.str.strip_chars_end(chars).alias("end"),
    addr.str.strip_chars_start(chars).alias("start"),
    addr.str.strip_prefix("128 ").alias("prefix"),
    addr.str.strip_suffix(", 158").alias("suffix"),
)

addresses,strip,end,start,prefix,suffix
str,str,str,str,str,str
"""128 PERF st""","""PERF st""","""128 PERF st""","""PERF st""","""PERF st""","""128 PERF st"""
"""Rust blVD, 158""","""Rust blVD""","""Rust blVD""","""Rust blVD, 158""","""Rust blVD, 158""","""Rust blVD"""
"""PoLaRs Av, 12""","""PoLaRs Av""","""PoLaRs Av""","""PoLaRs Av, 12""","""PoLaRs Av, 12""","""PoLaRs Av, 12"""
"""1042 Query sq""","""Query sq""","""1042 Query sq""","""Query sq""","""1042 Query sq""","""1042 Query sq"""


### Slicing

In [21]:
df = pl.DataFrame(
    {
        "fruits": ["pear", "mango", "dragonfruit", "passionfruit"],
        "n": [1, -1, 4, -4],
    }
)

df.with_columns(
    pl.col("fruits").str.slice(pl.col("n")).alias("slice"),
    pl.col("fruits").str.head(pl.col("n")).alias("head"),
    pl.col("fruits").str.tail(pl.col("n")).alias("tail"),
)

fruits,n,slice,head,tail
str,i64,str,str,str
"""pear""",1,"""ear""","""p""","""r"""
"""mango""",-1,"""o""","""mang""","""ango"""
"""dragonfruit""",4,"""onfruit""","""drag""","""ruit"""
"""passionfruit""",-4,"""ruit""","""passionf""","""ionfruit"""
